<a href="https://colab.research.google.com/github/lauragoon/Face-Mask-Detector/blob/main/svm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import keras
from tensorflow.keras import datasets, layers, models
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
# from keras.models import Model
# from keras import optimizers

from sklearn import svm
from sklearn.model_selection import GridSearchCV

from skimage.io import imread
from skimage.transform import resize

In [4]:
def get_data(batch_size, directory):
  
  datagen = image.ImageDataGenerator()
  dta = datagen.flow_from_directory(directory, class_mode='binary', batch_size=batch_size)

  return dta

In [5]:
## generate data
train_path = 'drive/Shared drives/DSCI 303 Final Project/ImageDataset2/training'
test_path = 'drive/Shared drives/DSCI 303 Final Project/ImageDataset2/test'

train_data = get_data(500, train_path)
test_data = get_data(100, test_path)

train_batch_images, train_batch_class = train_data.next()
test_batch_images, test_batch_class = test_data.next()

## pre-trained neural network to extract features
nn_model = ResNet50(input_shape=(256,256,3),
                    weights='imagenet',
                    include_top=False,
                    pooling='avg')

## feature extraction
img_features = nn_model.predict(train_batch_images).T

Found 7522 images belonging to 2 classes.
Found 1012 images belonging to 2 classes.
94773248/94765736 [==============================] - 1s 0us/step


In [6]:
flat_data = []
images = []
for i in train_batch_images:
  img_resized = resize(train_batch_images[0], (256, 256), anti_aliasing=True, mode='reflect')
  flat_data.append(img_resized.flatten()) 
flat_data = np.array(flat_data)


In [9]:
test_flat_data = []
test_images = []
for i in test_batch_images:
  img_resized = resize(test_batch_images[0], (256, 256), anti_aliasing=True, mode='reflect')
  test_flat_data.append(img_resized.flatten()) 
test_flat_data = np.array(test_flat_data)

In [10]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid)
clf.fit(flat_data, train_batch_class)

y_pred = clf.predict(test_flat_data)


In [12]:
num_data = len(y_pred)

accuracy = 0

for i in range(num_data):
    if y_pred[i] == test_batch_class[i]:
        accuracy += 1

print(accuracy / num_data)

0.92


In [ ]:
# reference: https://github.com/whimian/SVM-Image-Classification/blob/master/Image%20Classification%20using%20scikit-learn.ipynb